###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2021 Semester 1

## Assignment 1: Pose classification with naive Bayes


**Student ID(s):** 
Chuen Ley Low: 1078133
Hoan Tran: 


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [39]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score

In [19]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess():
    train_X = []
    train_Y = []
    train_file_path = "data/train.csv"
    test_file_path = "data/test.csv"
    
    train_df = pd.read_csv(train_file_path)
    test_df = pd.read_csv(test_file_path)
    
    train_X.append([float(x) for x in list(train_df.columns[1:])])
    train_Y.append(train_df.columns[0])
    
    for row in train_df.iterrows():
        temp = []
        for e in row[1][1:]:
            temp.append(e)
            
            
        train_X.append(temp)
        train_Y.append(row[1][0])
    
    test_X = []
    test_Y = []
    
    test_X.append([float(x) for x in list(test_df.columns[1:])])
    test_Y.append(test_df.columns[0])
    
    for row in test_df.iterrows():
        temp = []
        for e in row[1][1:]:
            temp.append(e)
            
            
        test_X.append(temp)
        test_Y.append(row[1][0])
    
    return train_X, train_Y, test_X, test_Y

train_X, train_Y, test_X, test_Y = preprocess()

In [24]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train():
    label_prob_dict = {}
    groups = {}
    for data, label in zip(train_X, train_Y):
        if label in groups:
            groups[label].append(data)
        else:
            groups[label] = [data]

    train_dict = {}
    
    for key in groups.keys():
        train_dict[key] = []
    
    for key in groups.keys():
        data = groups[key]
        num_attributes = len(data[0])
        num_instance = len(data)
        
        label_prob_dict[key] = num_instance / len(train_X)
        
        for i in range(num_attributes):
            total_sum = 0 
            for j in range(num_instance):
                if math.floor(data[j][i]) != 9999.0:
                    total_sum += float(data[j][i]) 
            
            mean = total_sum / num_instance
            
            diff_sum = 0
            
            for j in range(num_instance):
                if math.floor(data[j][i]) != 9999.0:
                    diff_sum += ((float(data[j][i]) - mean) ** 2)
                
            std_dev = math.sqrt(diff_sum / num_instance)
            
            train_dict[key].append((mean, std_dev))
            
    
    return train_dict, label_prob_dict

train_dict, label_prob_dict = train()        

{'bridge': 0.10843373493975904,
 'childs': 0.09236947791164658,
 'downwarddog': 0.13788487282463185,
 'mountain': 0.214190093708166,
 'plank': 0.07630522088353414,
 'seatedforwardbend': 0.05756358768406961,
 'tree': 0.08969210174029452,
 'trianglepose': 0.07898259705488621,
 'warrior1': 0.07228915662650602,
 'warrior2': 0.07228915662650602}

In [23]:
def GaussianFunction(x, mean, std_dev):
    x = float(x)
    base = 1 / (std_dev * math.sqrt(2 * math.pi))
    exponent = math.exp(-0.5 * ((x - mean)/std_dev) ** 2)
    return base * exponent

GaussianFunction(2.0, 1.0, 1.0)

0.24197072451914337

In [43]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict():
    predictions = []
    for i in range(len(test_X)):
        instance = test_X[i]
        prob_arr = []
        # Calculate the Gaussian Probability for each label
        for label in train_dict.keys():        
            prob = math.log(label_prob_dict[label], 2)
            for j in range(len(instance)):
                if math.floor(instance[j]) != 9999.0:
                    mean, std_dev = train_dict[label][j]
                    gaussian_prob = GaussianFunction(instance[j], mean, std_dev)
                    
                    if gaussian_prob != 0.0:
                        prob += math.log(GaussianFunction(instance[j], mean, std_dev), 2)
            prob_arr.append(prob)
        
        max_arg = np.argmax(prob_arr)
        predicted_label = list(train_dict.keys())[max_arg]
        predictions.append(predicted_label)
    
    print(predictions)
    return predictions

predictions = predict()

['bridge', 'childs', 'bridge', 'plank', 'downwarddog', 'bridge', 'bridge', 'bridge', 'childs', 'bridge', 'downwarddog', 'bridge', 'downwarddog', 'bridge', 'seatedforwardbend', 'childs', 'childs', 'mountain', 'childs', 'childs', 'seatedforwardbend', 'childs', 'childs', 'childs', 'childs', 'childs', 'childs', 'downwarddog', 'bridge', 'mountain', 'downwarddog', 'downwarddog', 'downwarddog', 'downwarddog', 'bridge', 'downwarddog', 'downwarddog', 'mountain', 'downwarddog', 'downwarddog', 'mountain', 'downwarddog', 'downwarddog', 'downwarddog', 'downwarddog', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'tree', 'tree', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'tree', 'tree', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'mountain', 'bridge', 'plank', 'plank', 'trianglepose', 'plank', 'plank', 'trianglepose', 'bridge', 'plank',

In [42]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate():
    acc_score = accuracy_score(test_Y, predictions)
    print(acc_score)

evaluate()

0.7413793103448276


## Questions 


If you are in a group of 1, you will respond to **two** questions of your choosing.

If you are in a group of 2, you will respond to **four** questions of your choosing.

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer should be submitted separately as a PDF.

### Q1
Since this is a multiclass classification problem, there are multiple ways to compute precision, recall, and F-score for this classifier. Implement at least two of the methods from the "Model Evaluation" lecture and discuss any differences between them. (The implementation should be your own and should not just call a pre-existing function.)

### Q2
The Gaussian naıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in this dataset? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the classifier’s predictions.

### Q3
Implement a kernel density estimate (KDE) naive Bayes classifier and compare its performance to the Gaussian naive Bayes classifier. Recall that KDE has kernel bandwidth as a free parameter -- you can choose an arbitrary value for this, but a value in the range 5-25 is recommended. Discuss any differences you observe between the Gaussian and KDE naive Bayes classifiers. (As with the Gaussian naive Bayes, this KDE naive Bayes implementation should be your own and should not just call a pre-existing function.)

### Q4
Instead of using an arbitrary kernel bandwidth for the KDE naive Bayes classifier, use random hold-out or cross-validation to choose the kernel bandwidth. Discuss how this changes the model performance compared to using an arbitrary kernel bandwidth.

### Q5
Naive Bayes ignores missing values, but in pose recognition tasks the missing values can be informative. Missing values indicate that some part of the body was obscured and sometimes this is relevant to the pose (e.g., holding one hand behind the back). Are missing values useful for this task? Implement a method that incorporates information about missing values and demonstrate whether it changes the classification results.

### Q6
Engineer your own pose features from the provided keypoints. Instead of using the (x,y) positions of keypoints, you might consider the angles of the limbs or body, or the distances between pairs of keypoints. How does a naive Bayes classifier based on your engineered features compare to the classifier using (x,y) values? Please note that we are interested in explainable features for pose recognition, so simply putting the (x,y) values in a neural network or similar to get an arbitrary embedding will not receive full credit for this question. You should be able to explain the rationale behind your proposed features. Also, don't forget the conditional independence assumption of naive Bayes when proposing new features -- a large set of highly-correlated features may not work well.